In [29]:
%load_ext autoreload
%autoreload 1
import sys; sys.path.append("..")
%aimport lib.util
from lib.util import reformat_tract, add_race_eth, add_purpose_type, add_income_group 
#add_all_category_columns
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1500
import numpy as np
from ambry import library
l = library()
p = l.get('ffiec.gov-hmda-2010e-lar-2012-ca').partition
df = p.select("SELECT * FROM lar WHERE county_code = 73").pandas

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
add_race_eth(df) # Adds race_eth column
add_purpose_type(df) # Adds purpose_type column Combines loan_purpose and loan_type in the same way as the summary report
add_income_group(df) # Adds income_group column
#add_all_category_columns(df) # Add '_c' categorical columns

In [56]:
def summaryTable(newdf, actiontype):
    totalcount = newdf.loan_amount.count()
    totalamount = newdf.loan_amount.sum()
    ### By Loan Purpose and Type
    loanpt = newdf.groupby(['purpose_type'])
    loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
    loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
    loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
    loanpt=loanpt[['count','%count','amount','%amount']]
    ### By Applicant Race/Ethnicity
    race_eth  = newdf.groupby(['race_eth'])
    race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
    race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
    race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
    race_eth = race_eth [['count','%count','amount','%amount']]
    ###By Applicant Income
    income  = newdf.groupby(['income_group'])
    income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
    income ['%count'] = np.round(100*income['count']/totalcount,1)
    income ['%amount'] = np.round(100*income['amount']/totalamount,1)
    income = income [['count','%count','amount','%amount']]
    ###By Applicant Sex
    sex  = newdf.groupby(['applicant_sex'])
    sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
    sex ['%count'] = np.round(100*sex['count']/totalcount,1)
    sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
    sex = sex [['count','%count','amount','%amount']]
    ###Owner-Occupied
    occ = newdf.groupby(['occupancy'])
    occ   = occ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
    occ ['%count'] = np.round(100*occ['count']/totalcount,1)
    occ ['%amount'] = np.round(100*occ['amount']/totalamount,1)
    occ = occ [['count','%count','amount','%amount']]
  
    ### Smashing Tables Together
    frames = [loanpt, race_eth,income,sex,occ]
    table = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex',
                                   'Occupancy'])
    pieces = [table.ix[:, ["count","%count","amount","%amount",]]]
    table = pd.concat(pieces, axis=1, keys=[actiontype])
    return table;

# All Lenders Combined

In [57]:
#Total -- All types
newdf=df
total=summaryTable(newdf,"Total")

#1 -- Loan originated
newdf = df[df.action_type==1]
action1 = summaryTable(newdf,"Loan originated")

#2 -- Application approved but not accepted
newdf = df[df.action_type==2]
action2 = summaryTable(newdf,"Application approved but not accepted")

#3 -- Application denied by financial institution
newdf = df[df.action_type==3]
action3 = summaryTable(newdf,"Application denied by financial institution")

#4 -- Application withdrawn by applicant
newdf = df[df.action_type==4]
action4 = summaryTable(newdf,"Application denied by financial institution")

#5 -- File closed for incompleteness
newdf = df[df.action_type==5]
action5 = summaryTable(newdf,"File closed for incompleteness")

#6 -- Loan purchased by the institution
newdf = df[df.action_type==6]
action6 = summaryTable(newdf,"Loan purchased by the institution")

#Merging All Tables
all_lenders = pd.concat([total,action1,action2,action3,action4,action5,action6], axis=1)
all_lenders

Total                    \
                                              count  %count    amount   
Loan Purpose and Type improvement              4781     2.0   1038687   
                      purchase-conventional   32857    13.5  12419787   
                      purchase-govt           22240     9.2   7551607   
                      refinance              182905    75.3  61055963   
Applicant Race/Eth    aian                      767     0.3    231319   
                      as                      23920     9.9   7808190   
                      b                        4053     1.7   1139822   
                      h                       24012     9.9   6385159   
                      pi                       1908     0.8    570463   
                      w                      142000    58.5  48384565   
Applicant Income      high                     1198     0.5   1344670   
                      low                    207208    85.3  64534660   
                      mid                      1557     0.6   1344339   
                      mod                      4278     1.8   2947579   
Applicant Sex         1                      159485    65.7  54930806   
                      2                       50394    20.8  14172171   
                      3                       17527     7.2   5939329   
                      4                       15377     6.3   7023738   
Occupancy             1                      208353    85.8  70232806   
                      2                       33252    13.7   9353423   
                      3                        1178     0.5   2479815   

                                                      Loan originated          \
                                             %amount            count  %count   
Loan Purpose and Type improvement                1.3             2315     1.7   
                      purchase-conventional     15.1            18452    13.7   
                      purchase-govt              9.2            10545     7.8   
                      refinance                 74.4           103541    76.8   
Applicant Race/Eth    aian                       0.3              417     0.3   
                      as                         9.5            14274    10.6   
                      b                          1.4             2219     1.6   
                      h                          7.8            12799     9.5   
                      pi                         0.7             1027     0.8   
                      w                         59.0            86821    64.4   
Applicant Income      high                       1.6              698     0.5   
                      low                       78.6           117924    87.4   
                      mid                        1.6              916     0.7   
                      mod                        3.6             2564     1.9   
Applicant Sex         1                         66.9            95760    71.0   
                      2                         17.3            29277    21.7   
                      3                          7.2             8844     6.6   
                      4                          8.6              972     0.7   
Occupancy             1                         85.6           115835    85.9   
                      2                         11.4            18108    13.4   
                      3                          3.0              910     0.7   

                                                                \
                                               amount  %amount   
Loan Purpose and Type improvement              624517      1.3   
                      purchase-conventional   7434158     16.0   
                      purchase-govt           3629798      7.8   
                      refinance              34805947     74.9   
Applicant Race/Eth    aian                     127627      0.3   
                      as                

In [45]:
all_lenders.to_csv('allLenders.csv')